In [7]:
import altair as alt
import json
import pandas as pd
import eco_style

alt.themes.enable('dark')

ThemeRegistry.enable('dark')

In [8]:
path = 'neet_youth_data.xlsx'
df = pd.read_excel(path, sheet_name = 'Sheet2')
df

,quarter,value,type
0,Jan-Mar 2002,365269.090796,Men
1,Apr-Jun 2002,371417.380979,Men
2,Jul-Sep 2002,372893.458240,Men
3,Oct-Dec 2002,343939.591604,Men
4,Jan-Mar 2003,375846.954840,Men
...,...,...,...
179,Oct-Dec 2023,391489.268808,Women
180,Jan-Mar 2024,406048.577978,Women
181,Apr-Jun 2024,395576.729609,Women
182,Jul-Sep 2024,403144.172314,Women


In [9]:


# Map each quarter string to its corresponding start month
quarter_to_month = {
    'Jan-Mar': '01',
    'Apr-Jun': '04',
    'Jul-Sep': '07',
    'Oct-Dec': '10'
}

def quarter_str_to_year_month(q_str):
    """
    Convert a string like 'Jan-Mar 2002' to '2002-01'.
    Handle any leading/trailing spaces.
    """
    # Remove leading/trailing whitespace
    q_str = q_str.strip()
    
    # Split into quarter_part (e.g., "Jan-Mar") and year_part (e.g., "2002")
    quarter_part, year_part = q_str.rsplit(' ', 1)
    
    # Map quarter abbreviation to month
    month = quarter_to_month[quarter_part]
    
    return f"{year_part}-{month}"

# Create a new column in the DataFrame with reformatted dates
df['year_month'] = df['quarter'].apply(quarter_str_to_year_month)

df


,quarter,value,type,year_month
0,Jan-Mar 2002,365269.090796,Men,2002-01
1,Apr-Jun 2002,371417.380979,Men,2002-04
2,Jul-Sep 2002,372893.458240,Men,2002-07
3,Oct-Dec 2002,343939.591604,Men,2002-10
4,Jan-Mar 2003,375846.954840,Men,2003-01
...,...,...,...,...
179,Oct-Dec 2023,391489.268808,Women,2023-10
180,Jan-Mar 2024,406048.577978,Women,2024-01
181,Apr-Jun 2024,395576.729609,Women,2024-04
182,Jul-Sep 2024,403144.172314,Women,2024-07


In [10]:
alt.themes.enable('light')

df['value'] = df['value']/1000000

chart = alt.Chart(df).transform_calculate(
        calculate='indexof(["Men", "Women"], datum.type)',
    as_= 'custom_order'	
).mark_area(interpolate='monotone').encode(
    x=alt.X('year_month:T', title=''),
    y=alt.Y('value:Q', title='', 
    axis=alt.Axis(labelExpr="format(datum.value,',') + 'M'")), 
    color=alt.Color('type:N', legend=None
                    , scale=alt.Scale(domain=["Men", "Women"], range=["#36B7B4", "#E6224B"]) 
    ),                      
    order='custom_order:Q',
    tooltip=['type:N', 'value:Q', 'year_month:T'] 
).properties(
    width=500,
    height=300,
    # title={
    #     "text": "UK: young people not in work or education",
    #     "fontSize" : 18,
    #     "anchor": "start",
    #     "subtitle": ["16 to 24 year-olds not in education, employment, or training", "Source: ONS 2025", ""],
    #       "subtitleColor": "#8F90AF"}
          )

text = alt.Chart(df).mark_text(
    align='left',
    size= 13,
    dx=3,
    dy= 
    {
  "expr": 
           "datum.type == 'Men' ? 50 : \
           datum.type == 'Women' ? -60 : \
           0"
}
).encode(
    alt.X('year_month:T', aggregate='max'),
    alt.Y('value:Q', aggregate={'argmax': 'year_month'}),
    text='type:N',
    color = 'type:N'
)  


chart2 = chart  + text
chart2



alt.LayerChart(...)

In [11]:
chart2.save('neet_youth.png', scale_factor=3)
chart2.save('neet_youth.svg')
chart2.save('neet_youth.json')